In [5]:
import requests
import urllib.request
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from string import punctuation
from heapq import nlargest
from math import log
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [6]:
def getWashPostText(url,token):
    try:
        page = urllib.request.urlopen(url).read().decode('utf8')
    except:
        return (None,None)
    soup = BeautifulSoup(page,"lxml")
    if soup is None:
        return (None,None)
    
    text = ""
    if soup.find_all(token) is not None:
       
        text = ''.join(map(lambda p: p.text, soup.find_all(token)))
        soup2 = BeautifulSoup(text,"lxml")
        if soup2.find_all('p')!=[]:
           
            text = ''.join(map(lambda p: p.text, soup2.find_all('p')))
    return text, soup.title.text

In [7]:
def getNYTText(url,token):
    response = requests.get(url)
    soup = BeautifulSoup(response.content,"lxml")
    page = str(soup)
    page
    title = soup.find('title').text
    mydivs = soup.findAll("p", {"class":"story-body-text story-content"})
    text = ''.join(map(lambda p:p.text, mydivs))
    return text, title

In [8]:
def scrapeSource(url, magicFrag='2018',scraperFunction=getNYTText,token='None'):
    urlBodies = {}
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    soup = BeautifulSoup(response,"lxml")

    numErrors = 0
    for a in soup.findAll('a'):
        try:
            url = a['href']
            
            if( (url not in urlBodies) and 
               ((magicFrag is not None and magicFrag in url) 
               or magicFrag is None)):
                body = scraperFunction(url,token)
                
                if body and len(body) > 0:
                    urlBodies[url] = body
                print("URL : " + str(url))
        except:
            numErrors += 1
            
    return urlBodies

In [9]:
urlWashingtonPostNonTech = "https://www.washingtonpost.com/sports/?utm_term=.3d9503255b93"
urlNewYorkTimesNonTech = "https://www.nytimes.com/section/sports"
urlWashingtonPostTech = "https://www.washingtonpost.com/business/technology/?utm_term=.2558fc9b8fdd"
urlNewYorkTimesTech = "https://www.nytimes.com/section/technology"

washingtonPostTechArticles = scrapeSource(urlWashingtonPostTech,
                                          '2018',
                                         getWashPostText,
                                         'article') 
print("\n\n\n")
washingtonPostNonTechArticles = scrapeSource(urlWashingtonPostNonTech,
                                          '2018',
                                         getWashPostText,
                                         'article')
print("\n\n\n")                                
newYorkTimesTechArticles = scrapeSource(urlNewYorkTimesTech,
                                       '2018',
                                       getNYTText,
                                       None)
print("\n\n\n")
newYorkTimesNonTechArticles = scrapeSource(urlNewYorkTimesNonTech,
                                       '2018',
                                       getNYTText,
                                       None)

URL : https://www.washingtonpost.com/news/worldviews/wp/2018/01/27/it-could-soon-be-a-crime-to-blame-poland-for-nazi-atrocities-and-israel-is-appalled/
URL : https://www.washingtonpost.com/news/acts-of-faith/wp/2018/01/28/hip-hop-is-having-a-god-moment-kendrick-lamar-is-a-big-part-of-it/?utm_term=.34044e117330
URL : https://www.washingtonpost.com/news/early-lead/wp/2018/01/25/lebron-james-drafted-a-much-better-all-star-team-than-stephen-curry-says-everyone/?utm_term=.73159c093a8b
URL : https://www.washingtonpost.com/news/worldviews/wp/2018/01/28/westerners-accused-of-singing-and-dancing-pornographically-could-face-a-year-in-cambodian-prison/
URL : https://www.washingtonpost.com/news/business/wp/2018/01/28/ikea-has-changed-the-way-we-think-about-furniture/
URL : https://www.washingtonpost.com/news/worldviews/wp/2018/01/28/the-desperate-attempt-to-save-frostbitten-climbers-on-one-of-the-worlds-highest-peaks/
URL : https://www.washingtonpost.com/news/retropolis/wp/2018/01/28/a-vietnam-war

URL : https://www.washingtonpost.com/news/early-lead/wp/2018/01/29/russell-westbrook-joel-embiid-feud-ramps-up-with-staredowns-and-a-monstrous-dunk/
URL : https://www.washingtonpost.com/news/nationals-journal/wp/2018/01/28/max-scherzer-receives-his-second-straight-nl-cy-young-award-at-bbwaa-dinner/
URL : https://www.washingtonpost.com/news/early-lead/wp/2018/01/28/the-pro-bowls-most-entertaining-moment-came-from-the-fightin-brees-boys/
URL : https://www.washingtonpost.com/news/early-lead/wp/2018/01/28/lavar-balls-lithuanian-head-coaching-debut-was-predictably-ridiculous/
URL : https://www.washingtonpost.com/sports/highschools/no-5-georgetown-prep-escapes-with-59-58-win-over-no-7-st-johns/2018/01/28/84ed27d6-01ef-11e8-bb03-722769454f82_story.html
URL : https://www.washingtonpost.com/news/sports/wp/2018/01/28/tom-izzo-says-michigan-state-will-cooperate-with-any-investigation-and-always-have/
URL : https://www.washingtonpost.com/news/soccer-insider/wp/2018/01/28/americans-abroad-alfredo-m

URL : https://www.nytimes.com/2018/01/28/sports/tennis/australian-open-roger-federer-marin-cilic.html
URL : https://www.nytimes.com/2018/01/28/sports/golf/tiger-woods-farmers-insurance-open.html
URL : https://www.nytimes.com/2018/01/28/sports/soccer/african-nations-championship.html
URL : https://www.nytimes.com/2018/01/26/sports/basketball/7-foot-7-robert-bobroczkyi-prospect.html
URL : https://www.nytimes.com/2018/01/28/sports/tennis/tennis-union.html
URL : https://www.nytimes.com/2018/01/28/sports/steve-yzerman-tampa-bay-lightning.html
URL : https://www.nytimes.com/2018/01/27/sports/basketball/thunder-dakari-johnson.html
URL : https://www.nytimes.com/2018/01/25/sports/football/malcolm-jenkins-eagles-super-bowl.html
URL : https://www.nytimes.com/2018/01/27/sports/tennis/sebastian-petr-korda-australian-open.html
URL : https://www.nytimes.com/2018/01/29/sports/football/the-big-game-isnt-as-super-for-its-hosts-as-is-thought.html
URL : https://www.nytimes.com/2018/01/27/sports/golf/tiger-

In [10]:
class FrequencySummarizer:
    def __init__(self,min_cut=0.1,max_cut=0.9):
        
        self._min_cut = min_cut
        self._max_cut = max_cut
        self._stopwords = set(stopwords.words('english') +
                              list(punctuation) +
                              [u"'s",'"'])
      
    def _compute_frequencies(self,word_sent,customStopWords=None):
        freq = defaultdict(int)
        if customStopWords is None:
            stopwords = set(self._stopwords)
        else:
            stopwords = set(customStopWords).union(self._stopwords)
        for sentence in word_sent:
            for word in sentence:
                if word not in stopwords:
                    freq[word] += 1
        m = float(max(freq.values()))
        for word in list(freq.keys()):
            freq[word] = freq[word]/m
            if freq[word] >= self._max_cut or freq[word] <= self._min_cut:
                del freq[word]
        return freq
    
    def extractFeatures(self,article,n,customStopWords=None):
        text = article[0]
        title = article[1]
        sentences = sent_tokenize(text)
        word_sent = [word_tokenize(s.lower()) for s in sentences]
        self._freq = self._compute_frequencies(word_sent,customStopWords)
        if n < 0:
            
            return nlargest(len(self._freq.keys()),self._freq,key=self._freq.get)
        else:
           
            return nlargest(n,self._freq,key=self._freq.get)

    def extractRawFrequencies(self, article):
 
        text = article[0]
        title = article[1]
        sentences = sent_tokenize(text)
        word_sent = [word_tokenize(s.lower()) for s in sentences]
        freq = defaultdict(int)
        for s in word_sent:
            for word in s:
                if word not in self._stopwords:
                    freq[word] += 1
        return freq
    
    def summarize(self, article,n):
        text = article[0]
        title = article[1]
        sentences = sent_tokenize(text)
        word_sent = [word_tokenize(s.lower()) for s in sentences]
        self._freq = self._compute_frequencies(word_sent)
        ranking = defaultdict(int)
        for i,sentence in enumerate(word_sent):
            for word in sentence:
                if word in self._freq:
                    ranking[i] += self._freq[word]
        sentences_index = nlargest(n,ranking,key=ranking.get)

        return [sentences[j] for j in sentences_index]

In [11]:
articleSummaries = {}
for techUrlDictionary in [newYorkTimesTechArticles, washingtonPostTechArticles]:
    for articleUrl in techUrlDictionary:
        if techUrlDictionary[articleUrl][0] is not None:
            if len(techUrlDictionary[articleUrl][0]) > 0:
                fs = FrequencySummarizer()
                summary = fs.extractFeatures(techUrlDictionary[articleUrl],25)
                articleSummaries[articleUrl] = {'feature-vector': summary,
                                               'label': 'Tech'}
for nontechUrlDictionary in [newYorkTimesNonTechArticles, washingtonPostNonTechArticles]:
    for articleUrl in nontechUrlDictionary:
        if nontechUrlDictionary[articleUrl][0] is not None:
            if len(nontechUrlDictionary[articleUrl][0]) > 0:
                fs = FrequencySummarizer()
                summary = fs.extractFeatures(nontechUrlDictionary[articleUrl],25)
                articleSummaries[articleUrl] = {'feature-vector': summary,
                                               'label': 'Non-Tech'}


In [12]:
def getDoxyDonkeyText(testUrl,token):
    response = requests.get(testUrl)
    soup = BeautifulSoup(response.content,"lxml")
    page = str(soup)
    title = soup.find("title").text
    mydivs = soup.findAll("div", {"class":token})
    text = ''.join(map(lambda p:p.text,mydivs))
    return text,title

testUrl = "http://doxydonkey.blogspot.in"
testArticle = getDoxyDonkeyText(testUrl,"post-body")

fs = FrequencySummarizer()
testArticleSummary = fs.extractFeatures(testArticle, 25)
testArticleSummary

['company',
 'china',
 'said',
 'new',
 'year',
 'billion',
 'uber',
 'million',
 'percent',
 'technology',
 'people',
 'could',
 'investors',
 'amazon',
 'drivers',
 'money',
 '1',
 'chief',
 'like',
 'india',
 'bitcoin',
 'last',
 'years',
 'pinterest',
 'raised']

In [13]:
# Using KNN

similarities = {}
for articleUrl in articleSummaries:
    oneArticleSummary = articleSummaries[articleUrl]['feature-vector']
    similarities[articleUrl] = len(set(testArticleSummary).intersection(set(oneArticleSummary)))

labels = defaultdict(int)    
knn = nlargest(5, similarities, key=similarities.get)
for oneNeighbor in knn:
    labels[articleSummaries[oneNeighbor]['label']] += 1

nlargest(1,labels,key=labels.get)

['Tech']

In [14]:
# Using Naive bayes Classifier

cumulativeRawFrequencies = {'Tech':defaultdict(int),'Non-Tech':defaultdict(int)}
trainingData = {'Tech':newYorkTimesTechArticles,'Non-Tech':newYorkTimesNonTechArticles}
for label in trainingData:
    for articleUrl in trainingData[label]:
        if len(trainingData[label][articleUrl][0]) > 0:
            fs = FrequencySummarizer()
            rawFrequencies = fs.extractRawFrequencies(trainingData[label][articleUrl])
            for word in rawFrequencies:
                cumulativeRawFrequencies[label][word] += rawFrequencies[word]


In [15]:
techiness = 1.0
nontechiness = 1.0
for word in testArticleSummary:
    if word in cumulativeRawFrequencies['Tech']:
        techiness *= 1e3*cumulativeRawFrequencies['Tech'][word] / float(sum(cumulativeRawFrequencies['Tech'].values()))
        
    else:
        techiness /= 1e3
        
    if word in cumulativeRawFrequencies['Non-Tech']:
        nontechiness *= 1e3*cumulativeRawFrequencies['Non-Tech'][word] / float(sum(cumulativeRawFrequencies['Non-Tech'].values()))
        
    else:
        nontechiness /= 1e3


techiness *= float(sum(cumulativeRawFrequencies['Tech'].values())) / (float(sum(cumulativeRawFrequencies['Tech'].values())) + float(sum(cumulativeRawFrequencies['Non-Tech'].values())))
nontechiness *= float(sum(cumulativeRawFrequencies['Non-Tech'].values())) / (float(sum(cumulativeRawFrequencies['Tech'].values())) + float(sum(cumulativeRawFrequencies['Non-Tech'].values())))
if techiness > nontechiness:
    label = 'Tech'
else:
    label = 'Non-Tech'
print(label, techiness, nontechiness)


Tech 1.9705342761825302e-10 1.8376525594285645e-28


In [16]:
def getAllDoxyDonkeyPosts(url,links):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    soup = BeautifulSoup(response,"lxml")
    for a in soup.findAll('a'):
        try:
            url = a['href']
            title = a['title']
            if title == "Older Posts":
                print(title, url)
                links.append(url)
                getAllDoxyDonkeyPosts(url,links)
        except:
            title = ""
    return

In [17]:
blogUrl = "http://doxydonkey.blogspot.in"
links = []
getAllDoxyDonkeyPosts(blogUrl,links)
doxyDonkeyPosts = {}
for link in links:
    doxyDonkeyPosts[link] = getDoxyDonkeyText(link,'post-body')


documentCorpus = []
for onePost in doxyDonkeyPosts.values():
    documentCorpus.append(onePost[0])


Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-05-23T19:53:00-07:00&max-results=7
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-05-14T19:02:00-07:00&max-results=7&start=7&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-05-02T19:43:00-07:00&max-results=7&start=14&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-04-17T19:26:00-07:00&max-results=7&start=21&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-04-10T18:56:00-07:00&max-results=7&start=28&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-03-30T19:57:00-07:00&max-results=7&start=35&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-03-20T19:47:00-07:00&max-results=7&start=42&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-03-02T17:42:00-08:00&max-results=7&start=49&by-date=false
Older Posts http://doxydonkey.bl

Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-04-23T20:19:00-07:00&max-results=7&start=462&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-04-14T19:40:00-07:00&max-results=7&start=469&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-04-05T20:22:00-07:00&max-results=7&start=476&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-03-24T20:12:00-07:00&max-results=7&start=483&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-03-15T20:41:00-07:00&max-results=7&start=490&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-03-03T19:30:00-08:00&max-results=7&start=497&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-02-22T19:55:00-08:00&max-results=7&start=504&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2015-02-11T20:02:00-08:00&max-results=7&start=511&by-date=false


In [18]:
# Using K-Means

vectorizer = TfidfVectorizer(max_df=0.5,min_df=2,stop_words='english')
X = vectorizer.fit_transform(documentCorpus)
km = KMeans(n_clusters = 5, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)
km.fit(X)

keywords = {}
for i,cluster in enumerate(km.labels_):
    oneDocument = documentCorpus[i]
    fs = FrequencySummarizer()
    summary = fs.extractFeatures((oneDocument,""),
                                100,
                                [u"according",u"also",u"billion",u"like",u"new", u"one",u"year",u"first",u"last"])
    if cluster not in keywords:
        keywords[cluster] = set(summary)
    else:
        keywords[cluster] = keywords[cluster].intersection(set(summary))

keywords

Initialization complete
Iteration  0, inertia 138.180
Iteration  1, inertia 71.669
Converged at iteration 1: center shift 0.000000e+00 within tolerance 7.274641e-09


{0: {'apple',
  'companies',
  'company',
  'facebook',
  'percent',
  'said',
  '—',
  '“',
  '”'},
 1: {'company', 'million', 'percent', 'said', 'would'},
 2: {'million', 'percent', 'said'},
 3: {'company', 'million', 'said', '“', '”'},
 4: {'said'}}